In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import time

Data will be from 1980-81 up to past season for train and validation splits. That is because prior to that season voting was done by players.

In [2]:
from collections import defaultdict

In [168]:
def work_player_profile(param, season):
    url = "https://www.basketball-reference.com" + param
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    advanced_table = soup.find(attrs={'id': 'all_advanced'})
    for child in advanced_table.children:
        if "table_outer_container" in child:
            other_soup = BeautifulSoup(child)
            rows = other_soup.findAll("tr")
    for row in rows:
        if 'id' in row.attrs and row.attrs['id'] == "advanced." + season:
            return {
                'per': float(row.find('td', attrs={'data-stat': 'per'}).text),
                'ts_pct': float(row.find('td', attrs={'data-stat': 'ts_pct'}).text),
                'usg_pct': float(row.find('td', attrs={'data-stat': 'usg_pct'}).text),
                'bpm': float(row.find('td', attrs={'data-stat': 'bpm'}).text),
            }
            
        

In [169]:
def get_stats_of_voting(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    item = soup.find(attrs={'class': 'stats_table'})
    rows = item.findAll("tr")
    
    season = url.split(".html")[0][-4:]
    
    players_stats = defaultdict(list)
    
    for row in rows:
        header_cells = row.findAll("th")
        for header_cell in header_cells:
            if 'data-stat' in header_cell.attrs and header_cell['data-stat'] == 'ranker' and 'csk' in header_cell.attrs:
                rank = int(header_cell.getText())
        td_cells = row.findAll("td")
        if not td_cells:
            continue
        for cell in td_cells:
            if 'data-stat' not in cell.attrs:
                continue
            if cell['data-stat'] == 'age' or cell['data-stat'] == 'team_id':
                continue
            if cell['data-stat'] == 'player':
                time.sleep(1)
                advanced_dict = work_player_profile(cell.find("a")['href'], season)
                for key in advanced_dict:
                    players_stats[key].append(advanced_dict[key])
                players_stats[cell['data-stat']].append(cell.getText())
            else:
                text = cell.getText() or "0"
                players_stats[cell['data-stat']].append(float(text))
    return players_stats

In [170]:
get_stats_of_voting("https://www.basketball-reference.com/awards/awards_2000.html")

defaultdict(list,
            {'per': [30.6,
              23.6,
              25.8,
              27.1,
              24.8,
              23.6,
              20.0,
              24.5,
              23.4,
              23.4,
              18.4,
              21.7,
              19.5,
              19.2,
              17.3,
              16.7],
             'ts_pct': [0.578,
              0.545,
              0.596,
              0.582,
              0.555,
              0.535,
              0.496,
              0.565,
              0.53,
              0.543,
              0.498,
              0.546,
              0.542,
              0.522,
              0.603,
              0.547],
             'usg_pct': [31.2,
              27.4,
              28.1,
              31.9,
              28.7,
              26.8,
              34.4,
              30.5,
              28.2,
              30.0,
              20.0,
              26.8,
              23.9,
              25.1,
              20.

In [ ]:
import re

def get_player_name_points(url, limit):
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    title = soup.find('h1').getText()
    splitted = re.split('\d{4}-\d{2}', title)
    name = splitted[0].strip()
    dash_splitted = title.split('-')
    season = dash_splitted[0].split(" ")[-1] + "-" + dash_splitted[1].split(" ")[0]
    player_names = list()
    seasons = list() # this will be filled with same values
    points = list()
    
    rows = soup.findAll('tr')
    for row in rows:
        cells = row.findAll('td')
        for cell in cells:
            if 'data-stat' in cell.attrs and cell['data-stat'] == 'reason':
                continue
            if 'data-stat' in cell.attrs and cell['data-stat'] == 'game_season':
                if cell.getText() == "":
                    continue
                if int(cell.getText()) > limit:
                    break
            if 'data-stat' in cell.attrs and cell['data-stat'] == 'pts':
                points.append(int(cell.getText()))
                player_names.append(name)
                seasons.append(season)
    
    return player_names, seasons, points